In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time

executable_path = '../driver/geckodriver'

driver = webdriver.Firefox(executable_path=executable_path)

driver.get('https://steamcommunity.com/app/435150/negativereviews/?p=1&browsefilter=toprated')

src_updated = driver.page_source
src = ""

for i in range(0,200):
    if src != src_updated:
        src = src_updated
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        src_updated = driver.page_source


# What can we get about the reviews from Steam?

# The review content, post date, how many people think this review helpful or funny

In [38]:
review_content=[]
usefuls=[]
gametimes=[]
userproducts=[]
recommends = []
usernames = []

for i in range(1,201):
    # get all Q&A list using XPATH locator
    lists=driver.find_elements_by_xpath("//div[@id='page%d']/div"%i)
#     print("//div[@id='page%d']/div"%i)
#     print("page%d pairs: "%i,len(lists))
    
    for idx,item in enumerate(lists):    
    # each Q&A pair has an unique ID
        div_id=item.get_attribute("id")
        content_css="div#"+div_id+" "+"div.apphub_UserReviewCardContent div.apphub_CardTextContent"
        useful_css="div#"+div_id+" "+"div.apphub_UserReviewCardContent div.found_helpful"
        gametime_css="div#"+div_id+" "+"div.apphub_UserReviewCardContent div.vote_header div.hours"
        userproduct_css="div#"+div_id+" "+"div.apphub_friend_block_container div.apphub_friend_block div.apphub_CardContentMoreLink"
        recommend_css="div#"+div_id+" "+"div.apphub_UserReviewCardContent div.vote_header div.title"
        username_css="div#"+div_id+" "+"div.apphub_friend_block_container div.apphub_friend_block div.apphub_CardContentAuthorName"
        
        review=driver.find_element_by_css_selector(content_css)
        useful=driver.find_element_by_css_selector(useful_css)
        gametime=driver.find_element_by_css_selector(gametime_css)
        userproduct=driver.find_element_by_css_selector(userproduct_css)
        recommend=driver.find_element_by_css_selector(recommend_css)
        username=driver.find_element_by_css_selector(username_css)
                
        review_content.append(review.text)
        usefuls.append(useful.text)
        gametimes.append(gametime.text)
        userproducts.append(userproduct.text)
        recommends.append(recommend.text)
        usernames.append(username.text)

print("Total reviews scraped: ", len(review_content), len(usefuls),len(gametimes),len(userproducts),len(recommends),len(usernames))
print(review_content[1])
print(usefuls[1])
print(gametimes[1])
print(userproducts[1])
print(recommends[1])
print(usernames[1])

Total reviews scraped:  757 757 757 757 757 757
Posted: October 10, 2017
Upfront, I'll say this game does have some fun gameplay at times. However a litany of problems make me give this a negative review.
- Dumbfoundingly inconsisent difficulty. Battles range from entertainingly fun to stupendously, insultingly difficult. Seriously, you should never have to go into a battle knowing EXACTLY what is going to happen in order to have a chance of surviving (not winning, just surviving). And the only reason you know what's about to happen is because you've tried it half a dozen times and failed each time.
- Weak story compared to D:OS, and heavy use of sad, tired and predictable tropes. Half the time it wasn't even clear what we were trying to accomplish, and who/what we were fighting against or for. And when major plot points did occur, most of the time our response was "Really? We're doing this again? Sigh..."
- For a game that encourages you to forge your own way, it punishes you for not 

In [23]:
# print(usefuls[0])
# print(usefuls[0].split("\n"))

2,003 people found this review helpful
32 people found this review funny
['2,003 people found this review helpful', '32 people found this review funny']


In [42]:
import pandas as pd
import nltk
import re

reviews =[]
dates = []
helpfuls = []
funnys = []
userproducts2 = []
spendtimes = []

for line in review_content:
    sentence = line.split("\n")
    date = re.sub(r"Posted:"," ",sentence[0]).strip()
    review = re.sub(r"Posted:.*"," ",line)
    review = re.sub(r"\s+"," ",review).strip() 
    reviews.append(review)
    dates.append(date)
    
print("date:",dates[0])
print("content:",reviews[0])

for line in usefuls:
    sentence = line.split("\n")
    
    if len(sentence)==2:
        helpful = re.findall(r"\d[,\d]*",sentence[0])
        if len(helpful)!=0:
            helpfuls.append(helpful[0])
        else:
            helpfuls.append("0")
        funny = re.findall(r"\d[,\d]*",sentence[1])
        funnys.append(funny[0])
        
    elif len(sentence)==1:
        helpful = re.findall(r"\d[,\d]*",sentence[0])
        if len(helpful)!=0:
            helpfuls.append(helpful[0])
        else:
            helpfuls.append("0")
        funnys.append("0")
    else:
        helpfuls.append("0")
        funnys.append("0")
        
print("helpful:",helpfuls[0])
print("funny:",funnys[0])
    
for line in gametimes:
    spendtime = re.findall(r"\d[.\d]*",line)
    spendtimes.append(spendtime[0])
    
print("time:",spendtimes[0])

for line in userproducts:
    product = re.findall(r"\d[.\d]*",line)
    userproducts2.append(product[0])
    
print("product:",userproducts2[0])

df = pd.DataFrame()
df["user_name"] = usernames
df["post_date"] = dates
df["helpful"] = helpfuls
df["funny"] = funnys
df["spent_time"] = spendtimes
df["recomment_or_not"] = recommends
df["user_product"] = userproducts2
df["review"] = reviews

df.to_csv('game_review_neg.csv',index=True)

date: February 26
content: I have never been more ambivalent about a game before in my life. I have never played a game with such amazing concepts and potential, squandered by fundamental failings of game design. The fine first, second, and third acts, while plagued with the issues I am going to discuss, are not nearly as horrible as the absolutely baffling experience of act 4. All issues previously felt in the game are so excacerbated in act 4, that I thought I was playing an entirely different game. 1. The armor system incentivizes perversely unfun tactics. With the recent changes of taunt, they have thankfully recognized this. Nonetheless, it causes both the players and the AI to initially launch into DEFCON 1 tier nuclear exchange, with the victor emerging as the one to freeze, knockdown, or terrify (fittingly, the most terrifying effect constantly abused by the AI) their opponent. When the encounters start becoming quite difficult, this exchange can pretty much decide the outcome 